In [3]:
from carla.data.catalog import OnlineCatalog
from carla.models.catalog import MLModelCatalog
from carla.models.negative_instances import predict_negative_instances
import carla.recourse_methods.catalog as recourse_catalog

from carla import Benchmark

import torch

dataset = OnlineCatalog("adult")

torch.manual_seed(0)

n_test = 10

ml_model = MLModelCatalog(
        dataset, 
        model_type="ann", 
        load_online=False, 
        backend="pytorch"
    )

ml_model.train(
    learning_rate=0.002,
    epochs=20,
    batch_size=1024,
    hidden_size=[18, 9, 3],
    force_train=True, 
)



balance on test set 0.23883245958934032, balance on test set 0.2408256880733945
Epoch 0/19
----------


/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


train Loss: 0.4668 Acc: 0.7734

test Loss: 0.4055 Acc: 0.8005

Epoch 1/19
----------
train Loss: 0.3946 Acc: 0.8121

test Loss: 0.3910 Acc: 0.8189

Epoch 2/19
----------
train Loss: 0.3784 Acc: 0.8222

test Loss: 0.3747 Acc: 0.8226

Epoch 3/19
----------
train Loss: 0.3655 Acc: 0.8290

test Loss: 0.3600 Acc: 0.8324

Epoch 4/19
----------
train Loss: 0.3535 Acc: 0.8343

test Loss: 0.3505 Acc: 0.8373

Epoch 5/19
----------
train Loss: 0.3460 Acc: 0.8372

test Loss: 0.3472 Acc: 0.8389

Epoch 6/19
----------
train Loss: 0.3431 Acc: 0.8387

test Loss: 0.3450 Acc: 0.8402

Epoch 7/19
----------
train Loss: 0.3405 Acc: 0.8402

test Loss: 0.3435 Acc: 0.8384

Epoch 8/19
----------
train Loss: 0.3404 Acc: 0.8389

test Loss: 0.3376 Acc: 0.8396

Epoch 9/19
----------
train Loss: 0.3348 Acc: 0.8421

test Loss: 0.3421 Acc: 0.8400

Epoch 10/19
----------
train Loss: 0.3348 Acc: 0.8411

test Loss: 0.3362 Acc: 0.8426

Epoch 11/19
----------
train Loss: 0.3345 Acc: 0.8401

test Loss: 0.3339 Acc: 0.8435



In [4]:
hyperparams = {
    "data_name": "adult",
    "batch_size": 1,
    "kappa": 0.1,
    "init_learning_rate": 0.01,
    "binary_search_steps": 9,
    "max_iterations": 100,
    "initial_const": 10,
    "beta": 0.9,
    "gamma": 1.0, # 0.0, #   1.0
    "mode": "PN",
    "num_classes": 2,
    "ae_params": {"hidden_layer": [20, 10, 7], "train_ae": True, "epochs": 5},
}

from tensorflow import Graph, Session

graph = Graph()
with graph.as_default():
    ann_sess = Session()
    with ann_sess.as_default():
        ml_model_sess = MLModelCatalog(dataset, "ann", "tensorflow") # data.catalog.DataCatalog Class

        factuals_sess = predict_negative_instances(
            ml_model_sess, dataset.df
        )
        factuals_sess = factuals_sess.iloc[:n_test].reset_index(drop=True)

        cem = recourse_catalog.CEM(ann_sess, ml_model_sess, hyperparams)
        df_cfs = cem.get_counterfactuals(factuals_sess)
        benchmark = Benchmark(ml_model, cem, factuals_sess)

distances = benchmark.compute_distances()

Train on 34182 samples, validate on 14650 samples
Epoch 1/5
34182/34182 [==============================] - 1s 33us/step - loss: 0.0658 - val_loss: 0.0258
Epoch 2/5
34182/34182 [==============================] - 1s 23us/step - loss: 0.0213 - val_loss: 0.0203
Epoch 3/5
34182/34182 [==============================] - 1s 21us/step - loss: 0.0173 - val_loss: 0.0181
Epoch 4/5
34182/34182 [==============================] - 1s 20us/step - loss: 0.0145 - val_loss: 0.0138
Epoch 5/5
34182/34182 [==============================] - 1s 21us/step - loss: 0.0116 - val_loss: 0.0108


In [ ]:
from carla import Benchmark

graph = Graph()
with graph.as_default():
    ann_sess = Session()
    with ann_sess.as_default():
        ml_model_sess = MLModelCatalog(dataset, "ann", "tensorflow") # data.catalog.DataCatalog Class

        factuals_sess = predict_negative_instances(
            ml_model_sess, dataset.df
        )
        factuals_sess = factuals_sess.iloc[:100].reset_index(drop=True)

        cem = recourse_catalog.CEM(ann_sess, ml_model_sess, hyperparams)
        benchmark = Benchmark(ml_model, cem, factuals_sess)


In [11]:
import pandas as pd
pd.set_option('display.max_columns', None)
benchmark._counterfactuals.iloc[0:1][dataset.continuous + ['sex_Male', 'marital-status_Non-Married', 'native-country_US', 'race_White', 'relationship_Non-Husband', 'sex_Male', 'workclass_Private']]

,age,fnlwgt,education-num,capital-gain,hours-per-week,capital-loss,sex_Male,marital-status_Non-Married,native-country_US,race_White,relationship_Non-Husband,sex_Male,workclass_Private
0,0.30137,0.044131,0.8,0.059,0.397959,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0


In [12]:
benchmark._factuals.iloc[0:1][dataset.continuous + ['sex_Male', 'marital-status_Non-Married', 'native-country_US', 'race_White', 'relationship_Non-Husband', 'sex_Male', 'workclass_Private']]

,age,fnlwgt,education-num,capital-gain,hours-per-week,capital-loss,sex_Male,marital-status_Non-Married,native-country_US,race_White,relationship_Non-Husband,sex_Male,workclass_Private
0,0.30137,0.044131,0.8,0.02174,0.397959,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0


In [22]:
import numpy as np

arr_f = ml_model.get_ordered_features(benchmark._factuals).to_numpy()
arr_cf = ml_model.get_ordered_features(
    benchmark._counterfactuals
).to_numpy()

delta = arr_f - arr_cf

d1 = np.sum(delta.round(2) != 0, axis=1, dtype=np.float).tolist()

In [23]:
d1

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [14]:
distances = benchmark.compute_distances()
distances.Distance_1

0    5.0
1    5.0
2    5.0
3    5.0
4    5.0
5    5.0
6    5.0
7    5.0
8    5.0
9    4.0
Name: Distance_1, dtype: float64

In [15]:
d1

[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0]